In [154]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import missingno as msno
import psycopg2
import getpass

In [155]:
# Request user for password to access database
db_password = getpass.getpass(prompt='DB Password ')

# Create SQL engine with connection parameters to connect to AWS RDS Postgres instance
engine = psycopg2.connect(
    database="Stores",
    user="postgres",
    password=db_password,
    host="stores-db.ck6vux0ulqkn.us-east-2.rds.amazonaws.com",
    port='5432'
)

DB Password ········


In [156]:
# Show sample of the imported data
train = pd.read_sql('SELECT * FROM store_table', engine)
train.head(1)

,store_id,store_area,items_available,daily_customer_count,store_sales
0,1,1659,1961,530,66490


In [157]:
train

,store_id,store_area,items_available,daily_customer_count,store_sales
0,1,1659,1961,530,66490
1,2,1461,1752,210,39820
2,3,1340,1609,720,54010
3,4,1451,1748,620,53730
4,5,1770,2111,450,46620
...,...,...,...,...,...
891,892,1582,1910,1080,66390
892,893,1387,1663,850,82080
893,894,1200,1436,1060,76440
894,895,1299,1560,770,96610


In [158]:
train.drop(train.columns[[0]], axis = 1, inplace = True)

In [159]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896 entries, 0 to 895
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   store_area            896 non-null    int64
 1   items_available       896 non-null    int64
 2   daily_customer_count  896 non-null    int64
 3   store_sales           896 non-null    int64
dtypes: int64(4)
memory usage: 28.1 KB


In [160]:
train.describe()

,store_area,items_available,daily_customer_count,store_sales
count,896.000000,896.000000,896.000000,896.000000
mean,1485.409598,1782.035714,786.350446,59351.305804
std,250.237011,299.872053,265.389281,17190.741895
min,775.000000,932.000000,10.000000,14920.000000
25%,1316.750000,1575.500000,600.000000,46530.000000
50%,1477.000000,1773.500000,780.000000,58605.000000
75%,1653.500000,1982.750000,970.000000,71872.500000
max,2229.000000,2667.000000,1560.000000,116320.000000


In [161]:
train.isnull().sum()

store_area              0
items_available         0
daily_customer_count    0
store_sales             0
dtype: int64

In [162]:
train.duplicated().sum()

0

In [163]:
for column_name in train.columns:
    unique_values = len(train[column_name].unique())
    print("Feature '{column_name}' has '{unique_values}' unique values".format(column_name = column_name,
                                                                                         unique_values=unique_values))

Feature 'store_area' has '583' unique values
Feature 'items_available' has '616' unique values
Feature 'daily_customer_count' has '130' unique values
Feature 'store_sales' has '816' unique values


In [170]:
train1=train
train2=train
train3=train
train.head()

,store_area,items_available,daily_customer_count,store_sales
0,1659,1961,530,66490
1,1461,1752,210,39820
2,1340,1609,720,54010
3,1451,1748,620,53730
4,1770,2111,450,46620


In [171]:
train4=train

In [172]:
train1=train1.drop('daily_customer_count',axis=1)
train2=train2.drop('daily_customer_count',axis=1)
train2=train2.drop('items_available',axis=1)
train3=train3.drop('daily_customer_count',axis=1)
train3=train3.drop('store_area',axis=1)
train4=train4.drop('store_area',axis=1)
train4=train4.drop('items_available',axis=1)

In [214]:
train4.head()

,daily_customer_count,store_sales,Out
0,530,66490,1
1,210,39820,-1
2,720,54010,1
3,620,53730,1
4,450,46620,1


In [177]:
from sklearn.neighbors import LocalOutlierFactor

In [178]:
clf = LocalOutlierFactor(n_neighbors=5, contamination='auto')
y_pred = clf.fit_predict(train)
y_pred1 = clf.fit_predict(train1) 
y_pred2 = clf.fit_predict(train2) 
y_pred3 = clf.fit_predict(train3)
y_pred4 = clf.fit_predict(train4)

In [179]:
train['Out']=y_pred
train1['Out']=y_pred1
train2['Out']=y_pred2
train3['Out']=y_pred3
train4['Out']=y_pred4

In [180]:
Out=train[train['Out']!=1]
Out1=train1[train1['Out']!=1]
Out2=train2[train2['Out']!=1]
Out3=train3[train3['Out']!=1]
Out4=train4[train4['Out']!=1]

In [185]:
Out4.shape

(47, 3)

In [186]:
train5=train[train['Out']==1]
train6=train1[train1['Out']==1]
train7=train2[train2['Out']==1]
train8=train3[train3['Out']==1]
train9=train4[train4['Out']==1]

In [208]:
train5

,store_area,items_available,daily_customer_count,store_sales,Out
0,1659,1961,530,66490,1
1,1461,1752,210,39820,1
2,1340,1609,720,54010,1
3,1451,1748,620,53730,1
4,1770,2111,450,46620,1
...,...,...,...,...,...
891,1582,1910,1080,66390,1
892,1387,1663,850,82080,1
893,1200,1436,1060,76440,1
894,1299,1560,770,96610,1


In [195]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [196]:
y=train5['store_sales']
X=train5.drop(['store_sales','Out'],axis=1)
y1=train6['store_sales']
X1=train6.drop(['store_sales','Out'],axis=1)
y2=train7['store_sales']
X2=train7.drop(['store_sales','Out'],axis=1)
y3=train8['store_sales']
X3=train8.drop(['store_sales','Out'],axis=1)
y4=train9['store_sales']
X4=train9.drop(['store_sales','Out'],axis=1)

In [197]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train1, X_test1, y_train1, y_test1=train_test_split(X1,y1,test_size=0.2,random_state=42)
X_train2, X_test2, y_train2, y_test2=train_test_split(X2,y2,test_size=0.2,random_state=42)
X_train3, X_test3, y_train3, y_test3=train_test_split(X3,y3,test_size=0.2,random_state=42)
X_train4, X_test4, y_train4, y_test4=train_test_split(X4,y4,test_size=0.2,random_state=42)

In [203]:
models = [LinearRegression(),RandomForestRegressor(),xgb.XGBRegressor()]
scores = dict()

for m in models:
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)
    

    print(f'model: {str(m)}')
    print(f'MAPE: {np.mean(np.abs((y_test - y_pred) / y_test)) * 100}')
    print('-'*30, '\n')
    

model: LinearRegression()
MAPE: 26.52514754089043
------------------------------ 

model: RandomForestRegressor()
MAPE: 29.13029113729928
------------------------------ 

model: XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)
MAPE: 30.811857416512677
------------------------------ 



In [204]:
for m in models:
    m.fit(X_train1, y_train1)
    y_pred1 = m.predict(X_test1)
    

    print(f'model: {str(m)}')
    print(f'MAPE: {np.mean(np.abs((y_test1 - y_pred1) / y_test1)) * 100}')
    print('-'*30, '\n')

model: LinearRegression()
MAPE: 28.2873364544741
------------------------------ 

model: RandomForestRegressor()
MAPE: 30.803879040144842
------------------------------ 

model: XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)
MAPE: 32.00037158712647
------------------------------ 



In [205]:
for m in models:
    m.fit(X_train2, y_train2)
    y_pred2 = m.predict(X_test2)
    

    print(f'model: {str(m)}')
    print(f'MAPE: {np.mean(np.abs((y_test2 - y_pred2) / y_test2)) * 100}')
    print('-'*30, '\n')

model: LinearRegression()
MAPE: 27.449477906823578
------------------------------ 

model: RandomForestRegressor()
MAPE: 33.036547683592744
------------------------------ 

model: XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)
MAPE: 34.24702931942542
------------------------------ 



In [206]:
for m in models:
    m.fit(X_train3, y_train3)
    y_pred3 = m.predict(X_test3)
 

    print(f'model: {str(m)}')
    print(f'MAPE: {np.mean(np.abs((y_test3 - y_pred3) / y_test3)) * 100}')
    print('-'*30, '\n')

model: LinearRegression()
MAPE: 25.431670451961804
------------------------------ 

model: RandomForestRegressor()
MAPE: 28.96170823297446
------------------------------ 

model: XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)
MAPE: 28.811828492976016
------------------------------ 



In [207]:
for m in models:
    m.fit(X_train4, y_train4)
    y_pred4 = m.predict(X_test4)
    s=m.score(X_train4,y_train4)

    print(f'model: {str(m)}')
    print(f'MAPE: {np.mean(np.abs((y_test4 - y_pred4) / y_test4)) * 100}')
    print('-'*30, '\n')

model: LinearRegression()
MAPE: 27.584902648385214
------------------------------ 

model: RandomForestRegressor()
MAPE: 29.928257112496432
------------------------------ 

model: XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)
MAPE: 30.425695729934098
------------------------------ 

